# Miniproject 2

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
import sys
import math
import numpy as np
from collections import namedtuple
import matplotlib.pyplot as plt

Pose = namedtuple("Pose", ["x", "y", "theta"])
Landmark = namedtuple("Landmark", ["x", "y"])
Measurement = namedtuple("Measurement", ["r", "b"])

def eval_gaussian(x, mean=0, var=1):
  return 1.0/math.sqrt(2*math.pi*var)*math.exp(-0.5*(x-mean)**2/var)

class Simulator:
  def __init__(self):
    np.random.seed()

    self.robot = Pose(0, 0, 0)

    self.motion_noise_covariance = [[1, 0], [0, 1]]
    self.sensor_noise_covariance = [[1, 0], [0, np.deg2rad(5)**2]]

    self.landmarks = [Landmark(-1, -1), Landmark(-1, -1), Landmark(11, 11), Landmark(-1, 11)]


  def reset(self):
    self.robot = Pose(0, 0, 0)

  def simulate_motion(self, delta_p, delta_theta):
    noise = np.random.multivariate_normal([0, 0], self.motion_noise_covariance)
    x = self.robot.x + math.cos(self.robot.theta)*(delta_p + noise[0])
    y = self.robot.y + math.sin(self.robot.theta)*(delta_p + noise[0])
    theta = self.robot.theta + delta_theta + noise[1]
    self.robot = Pose(x, y, theta)

  def simulate_sensing(self):
    measurements = []
    for landmark in self.landmarks:
      noise = np.random.multivariate_normal([0, 0], self.sensor_noise_covariance)

      r = math.hypot(landmark[0]-self.robot.x, landmark[1]-self.robot.y) + noise[0]
      b = math.atan2(landmark[1]-self.robot.y, landmark[0]-self.robot.x) + noise[1]

      measurements.append(Measurement(r, b))

    return measurements

def drive_in_a_square(simulator):
  poses = []
  poses.append(simulator.robot)
  for sides in range(4):
    for i in range(100):
      simulator.simulate_motion(.1, 0)
      poses.append(simulator.robot)
    simulator.simulate_motion(0, np.pi/2)
    poses.append(simulator.robot)

  return np.array(poses)

def press(event):
  if event.key == 'q':
    sys.exit(0)
  plt.close()

def draw_samples(samples):
  fig, ax = plt.subplots()
  ax.axis([-2, 12, -2, 12])
  fig.canvas.mpl_connect('key_press_event', press)
  ax.plot(samples[:, 0], samples[:, 1], 'g+')
  plt.show()

def draw_trajectories(true_poses, estimated_poses):
  fig, ax = plt.subplots()
  ax.axis([-20, 20, -20, 20])
  fig.canvas.mpl_connect('key_press_event', press)

  ax.plot(true_poses[:, 0], true_poses[:,1], 'b')
  ax.plot(estimated_poses[:, 0], estimated_poses[:,1], 'r')
  plt.show()




## Problems

### Warmup 1
Let's make sure we know how to resample a set of
weights.

Please write a function that takes in a list of samples and associated weights,
and re-samples from the list of samples with replacement according the
likelihood of each sample given by the list of weights. Please generate 100 samples.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy.


For reference, our solution is **1** lines of code.

In [ ]:
def warmup_1(samples, weights):
  raise NotImplementedError("Implement me!")

Tests

In [ ]:

print('Tests passed.')

### Warmup 2
We are going to be sampling from Gaussian
systems in this miniproject. Let us check that we know how to sample from a
multi-variate Gaussian.

Please write a function that takes in a mean vector, a covariance matrix and a
integer number of samples and returns a list of the requested number of
samples drawn from a Gaussian with the given mean and covariance.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy.


For reference, our solution is **2** lines of code.

In [ ]:
def warmup_2(mean, covariance, num_samples):
  """Sample from a multi-variate Gaussian distribution.

  Args:
    mean: a vector in numpy.
    covariance: a matrix in numpy.
    num_samples: an integer.

  Returns:
    sample: a numpy vector with length num_samples.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:

print('Tests passed.')

### Sampling Question 1
Let us now start by building a motion model for a
quasi-holonomic ground robot.

A common motion model for such a robot assumes that the robot has a pose given
by $[x_t, y_t, \theta_t]$ at time $t$. The robot takes a command $[\Delta p_t,
\Delta \theta_t]$ which modifies the robot's pose accoding to the following:

$$
\begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \end{bmatrix} =
\begin{bmatrix} x_t + \cos(\theta_t) (\Delta p_t + \omega_t) \\
y_t + \sin(\theta_t)(\Delta p_t + \omega_t) \\
\theta_t + \Delta \theta_t + \nu_t
\end{bmatrix}
$$

where $[\omega_t, \nu_t]$ is Gaussian error injected at each time
step, distributed as $\omega_t \sim \mathcal{N}(0, \sigma^2_p)$, and
$\nu_t \sim \mathcal{N}(0, \sigma^2_\theta)$. The units of $\Delta  p_t$ is in
metres and $\Delta \theta_t$ in radians.

Let us assume that the robot's pose at each time step is a random variable
given as a set of samples.

Please implement a function that takes in a set of sampled poses
drawn from prior pose random variable $\mathbf{X}_t$
and a control $[\Delta p_t, \Delta \theta_t]$ and returns a set
of samples drawn from the posterior $X_{t+1}$.



For reference, our solution is **13** lines of code.

In [ ]:
def motion_model(samples, delta_p, delta_theta, simulator):
  """A motion model.

  Args:
    samples: a list of samples
    delta_p: translation command
    delta_theta: rotation command
    simulator: a Simulator object

  Returns:
    a list of new samples by applying the motion model
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:

print('Tests passed.')

### Sampling Question 2
Let us now add in a measurement model. Let us assume that
there are four landmarks located at locations (-1, -1), (11, -1), (11, 11) and
(-1, 11). The robot is equipped with a sensor that can measure the range and
bearing [r, b] from the robot's location to each of these landmarks. Each of
these measurements is corrupted with noise $[q_r, q_b]$ distributed according
to $q_r \sim \mathcal{N}(0, \sigma^2_r)$ and $q_b \sim \mathcal{N}(0,
\sigma^2_b)$.

Given a set of measurements, we can use this model to compute the importance
weights of a set of samples of the robot pose.

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples.



For reference, our solution is **18** lines of code.

In [ ]:
def compute_weights(samples, measurement, simulator):
  """Compute the importance weights of the samples.

  Args:
    a list of samples
    a measurement vector
    a simulator object

  Returns:
    a list of importance weights
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:

print('Tests passed.')

### Main Question
Now let's put everything together. Please write a method
localization() that does the following:

  * Creates a simulator.
  * Initialize a set of samples drawn from a Gaussian distribution with mean
equal to the robot starting position, and a diagonal covariance matrix equal
to diag(1, 1, np.deg2rad(5))^2.
  * Drive the robot along the same pattern as in the drive_in_a_square method,
but on each step, after the command motion do the following:
  * use the motion_model method to update the sample between each simulated
motion,
  * use compute_weights to compute the sample weights from the
measurements,
  * resample a new set of samples,
  * blur the samples by applying noise drawn from a zero-mean Gaussian with
covariance equal to `diag(.01, .01, np.deg2rad(1))^2`.


For reference, our solution is **38** lines of code.

In [ ]:
def localization(simulator):
  """Main function for robot localization.

  Args:
    simulator: a Simulator object.

  Returns:
    true_poses
    estimated_poses
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:

print('Tests passed.')